In [135]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [136]:
df = pd.read_csv('banana_dataset.csv')
print(df.head(5))

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [ ]:
def synthesize_ethylene(gas):
    ethylene_baseline = 50
    ethylene_increase_rate = 0.5
    peak_gas = 350
    decline_rate = 0.1
    
    if gas < peak_gas:
        return ethylene_baseline + (gas * ethylene_increase_rate)
    else:
        return ethylene_baseline + (peak_gas * ethylene_increase_rate) - ((gas - peak_gas) * decline_rate)


In [ ]:
df['Synthesized Ethylene'] = df['Ethanol (Gas)'].apply(synthesize_ethylene)
df

In [ ]:
df["Grade"] = [0 for i in range (2245)]

# Grade 1: Unripe
df.loc[(df['Ethylene'] < 130) & (df['Ethanol (Gas)'] < 130), 'Grade'] = 1

# Grade 2: Ripe (perfect to eat)
df.loc[((df['Ethylene'] >= 130) & (df['Ethylene'] < 180)) & ((df['Ethanol (Gas)'] >= 130) & (df['Ethanol (Gas)'] < 210)), 'Grade'] = 2

# Grade 3: Over ripe
df.loc[((df['Ethylene'] >= 180) & (df['Ethylene'] < 200)) & ((df['Ethanol (Gas)'] >= 210) & (df['Ethanol (Gas)'] < 360)), 'Grade'] = 3

# Grade 4: Rotting stage
df.loc[(df['Ethylene'] >= 200) | (df['Ethanol (Gas)'] >= 360), 'Grade'] = 4

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [138]:
df_drop = df.drop(['created_at','entry_id' ], axis=1)

KeyError: "['created_at', 'entry_id'] not found in axis"

In [ ]:
df_new = df_drop.dropna()

In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2231 entries, 0 to 2244
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Temperature    2231 non-null   float64
 1   Humidity       2231 non-null   float64
 2   Ethanol (Gas)  2231 non-null   float64
 3   Type of Fruit  2231 non-null   object 
 4   Ethylene       2231 non-null   float64
 5   Grade          2231 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 122.0+ KB


In [ ]:
X = df_new[['Ethanol (Gas)', 'Ethylene']]
y = df_new['Grade'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(2,))
])

c:\Users\USER\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

Epoch 1/50


157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0467 - loss: 481.0732  
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0537 - loss: 223.5563
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0528 - loss: -18.7117
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1307 - loss: -211.8413  
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1390 - loss: -380.3779
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1598 - loss: -563.5396 
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1500 - loss: -774.6818   
Epoch 8/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step - accuracy: 0.1588 - loss: -931.4358
Epoch 9/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1341 - loss: -1117.5024
Epoch 10/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1519 - loss: -1302.1083
Epoch 11/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1365 - loss: -1507.6938
Epoch 12/50
157/157 